# CS F425- Deep Learning Project
Implementation of CNN for Digit Recognition

TODO-
1.   Expand on Dataset to enable alphabet recognition
2.   Modify activation functions/number of layers for higher accuracy 
3.   Optimize tradeoff for higher resolution while maintaining performance
4.   Simplify test process using Tk



In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import image
from PIL import Image
import cv2

#import warnings
#warnings.filterwarnings('ignore')

In [ ]:
#Load data from MNIST
handwriting_mnist = keras.datasets.mnist
(train_images,train_labels),(test_images,test_labels) = handwriting_mnist.load_data()
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [ ]:
# Test Data Visualization
fig = plt.figure(figsize=(8, 8))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

# plot the digits: each image is 28x28 pixels
for i in range(64):
    ax = fig.add_subplot(8, 8, i + 1, xticks=[], yticks=[])
    ax.imshow(test_images[i].reshape(28,28), cmap=plt.cm.binary, interpolation='nearest')
    
    # label the image with the target value
    ax.text(0, 7, str(test_labels[i]))
plt.show()

In [4]:
# Creating model
model = keras.models.Sequential([
                                 keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)),
                                 keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2,2)),
                                 keras.layers.Conv2D(64,(3,3),activation='relu'),
                                 keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2,2)),
                                 keras.layers.Flatten(),
                                 keras.layers.Dense(128,activation='relu'),
                                 keras.layers.Dense(10,activation='softmax')
                                ])

print("=> Model Summary-")
print(model.summary())

=> Model Summary-
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 12

In [5]:
# Model Compile
model.compile(optimizer='adam',metrics=['acc'],loss='sparse_categorical_crossentropy')

In [6]:
# Accuracy Callback
class handwriting_acc_callback(keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs={}):
    if(logs.get('acc') >= 0.9):
       print("\n Model has reached 90% accuracy!")
       self.model.stop_training = True

handwriting_acc_callback = handwriting_acc_callback()

# Model Fitting
print("=> Fitting the Model-")
model.fit(train_images,train_labels,epochs=100,callbacks=[handwriting_acc_callback])

# Model Evaluation
print("=> Model Evaluation-")
model.evaluate(test_images,test_labels)

=> Fitting the Model-
Epoch 1/100
1874/1875 [============================>.] - ETA: 0s - loss: 0.3246 - acc: 0.9492
 Model has reached 90% accuracy!
1875/1875 [==============================] - 26s 13ms/step - loss: 0.3245 - acc: 0.9492
=> Model Evaluation-
313/313 [==============================] - 1s 4ms/step - loss: 0.0761 - acc: 0.9772


[0.07613489031791687, 0.9771999716758728]

Data Source: https://keras.io/datasets/#mnist-database-of-handwritten-digits

In [7]:
from google.colab import files# Model Prediction
print("=> Model Prediction-")
uploaded = files.upload()

for fn in uploaded.keys():
  path = '/content/' + fn
  img = image.load_img(path)
  x = image.img_to_array(img)
  x = cv2.cvtColor(x, cv2.COLOR_BGR2GRAY)
  print("**************** Uploaded Image ****************")
  plt.imshow(x, cmap='gray')
  plt.show()
  x = cv2.resize(np.array(x), (28, 28))
  x = x.reshape(28, 28, 1)
  x = np.expand_dims(x, axis=0)

  class_label = model.predict(x)
  print("Predicted Value is:",np.where(class_label[0] == max(class_label[0]))[0])

ModuleNotFoundError: No module named 'google.colab'

Here, I am trying to use tkinter so that we can draw the input image instead of uploading it as a file in the above block. 

NOTE- This will NOT run on google collab. Try it by importing this notebook and running natively:

In [ ]:
from PIL import ImageTk, ImageDraw
import PIL
from tkinter import *

width = 200 
height = 200 
center = height//2
white = (255, 255, 255) 

def save():
    filename = "input.jpg"
    output_image.save(filename)

def paint(event):
    x1, y1 = (event.x - 1), (event.y - 1)
    x2, y2 = (event.x + 1), (event.y + 1)
    canvas.create_oval(x1, y1, x2, y2, fill="black",width=5)
    draw.line([x1, y1, x2, y2],fill="black",width=5)

master = Tk()

canvas = Canvas(master, width=width, height=height, bg='white')
canvas.pack()

output_image = PIL.Image.new("RGB", (width, height), white)
draw = ImageDraw.Draw(output_image)
canvas.pack(expand=YES, fill=BOTH)
canvas.bind("<B1-Motion>", paint)

button=Button(text="save",command=save)
button.pack()

master.mainloop()